In [1]:
!pip install transformers
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
"""
import shutil
shutil.rmtree("./mlabelBERToverall")
shutil.rmtree("./multilabeltokenizer")
import os 
os.remove("./mlabelBERT.zip")
os.remove("./y_pred.npy")
"""

In [ ]:
# Explainability with SHAP
# Reference: https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/text.html

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import warnings
import tensorflow_addons as tfa
import spacy
warnings.filterwarnings("ignore",category=DeprecationWarning)
pd.set_option('max_colwidth', 800)
pd.set_option('max_columns', 50)

# Function for text cleaning

In [ ]:
# Data Cleaning (Lowering, Removing stopwords)
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
#stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def cleaning(data):

   corpus = []
   for i in range(0, len(data)):
     sentence = re.sub('[^a-zA-Z]', ' ', str(data[i]))
     sentence = sentence.lower()
     sentence = sentence.split()
     all_stopwords = stopwords.words('english')
     all_stopwords.extend(['from', 'subject','tw', 're','[removed]','hi' 'edu','reddit','subreddit','account','mod','hi','im','w','h','ide','gary',
                           'mayb'])
     #all_stopwords.remove('not')
     sentence = [word for word in sentence if not word in set(all_stopwords)]
     sentence = ' '.join(sentence)
     corpus.append(sentence)
      
   return corpus


In [ ]:
def lemmatization(texts, allowed_postags=["NOUN","ADV", "VERB"  "ADJ"]):    # "ADV", "VERB"  "ADJ"
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

# Data loading and preparation

In [ ]:
data = pd.read_csv("../input/csa-dataset/total_csaws.csv")
data['cleaned_text'] = cleaning(data['Text'])
"""
data['cleaned_text'] = lemmatization(data['cleaned_text'])
"""
print(data.shape)
data.head(1)

(7957, 11)


,subreddit,id,author,num_comments,title,selftext,Text,dep,anx,ptsd,cleaned_text
0,adultsurvivors,t1r60p,baybecay,1,"""Only sex"" - poem I made while coping with depression/feeling empty TW graphic","Im not sure if this kind of art post is against the rules if so apologies to the mods. \n\nI forgot my earliest memory \n\nCame back later\n\nI was being raped on the bathroom floor \n\nOr maybe it was in my bed\n\nOr maybe on my bedroom floor \n\nIm not sure in what order\n\nBut my deepest feelings hurt\n\nThe void of love \n\nFilled with pain in my chest \n\nInescapable no matter how high, \n\nhow delirious,\n\nhow deep into psychosis I go\n\n\nMy pain is dirty\n\nIts pull my hair while you anally rape me pain \n\nHave your friend slap me after he pays you \n\nFor the rough stuff pain \n\nHold a knife to my 5 year old throat\n\nThreaten to kill me pain \n\nShove the barrel of your gun into my pussy pain\n\nCut her head of and rape me next to her body\n\non camera pain\n\nTrain my bod...","""Only sex"" - poem I made while coping with depression/feeling empty TW graphic Im not sure if this kind of art post is against the rules if so apologies to the mods. \n\nI forgot my earliest memory \n\nCame back later\n\nI was being raped on the bathroom floor \n\nOr maybe it was in my bed\n\nOr maybe on my bedroom floor \n\nIm not sure in what order\n\nBut my deepest feelings hurt\n\nThe void of love \n\nFilled with pain in my chest \n\nInescapable no matter how high, \n\nhow delirious,\n\nhow deep into psychosis I go\n\n\nMy pain is dirty\n\nIts pull my hair while you anally rape me pain \n\nHave your friend slap me after he pays you \n\nFor the rough stuff pain \n\nHold a knife to my 5 year old throat\n\nThreaten to kill me pain \n\nShove the barrel of your gun into my pussy pain\n\...",1,0,0,sex poem made coping depression feeling empty graphic sure kind art post rules apologies mods forgot earliest memory came back later raped bathroom floor maybe bed maybe bedroom floor sure order deepest feelings hurt void love filled pain chest inescapable matter high delirious deep psychosis go pain dirty pull hair anally rape pain friend slap pays rough stuff pain hold knife year old throat threaten kill pain shove barrel gun pussy pain cut head rape next body camera pain train body bend every pleasure pain call babycakes camera sometimes buttercup sunflower tie hit slap shock burn loved eyes judgement felt worthless unloved devolved humiliated intensely afriad knew take addicted meth shoved throat equally addicted loved felt warmth freshly abused year old pussy vibrating whole body ...


# Text Preprocessing

In [5]:
data = pd.read_excel("Predictions.xlsx")
data = data[['title_text', 'Final Annotation Majority Voting Criteria']]
data = data.iloc[1:]
data = data.dropna()
print(data.shape[0])
data_train = data.iloc[0:91][:]
data_test = data[91:][:]
print(data_train.shape)
print(data_test.shape)
data['num_words'] = data.title_text.apply(lambda x:len(x.split()))
print(data.num_words.max())
labels = ['belongingness','safety','esteem','self-actualization','physiological']
label_df = []
for i in range(data.shape[0]):
    lab = []
    title_text = data.iloc[i][1]
    if title_text.find(labels[0])!=-1:
        lab.append(1.0)
    else:
        lab.append(0.0)

    if title_text.find(labels[1])!=-1:
        lab.append(1.0)
    else:
        lab.append(0.0)

    if title_text.find(labels[2])!=-1:
        lab.append(1.0)
    else:
        lab.append(0.0)
    
    if title_text.find(labels[3])!=-1:
        lab.append(1.0)
    else:
        lab.append(0.0)

    if title_text.find(labels[4])!=-1:
        lab.append(1.0)
    else:
        lab.append(0.0)
    label_df.append(lab)
belong = []
safety = []
esteem = []
self_ = []
phy = []
for i in range(len(label_df)):
    lab = label_df[i]
    belong.append(lab[0])
    safety.append(lab[1])
    esteem.append(lab[2])
    self_.append(lab[3])
    phy.append(lab[4])
data["belongingness"] = belong
data["safety"] = safety
data["esteem"] = esteem
data["self_actualization"] = self_
data["physiological"] = phy
data.head()

114
(91, 2)
(23, 2)
136


,title_text,Final Annotation Majority Voting Criteria,num_words,belongingness,safety,esteem,self_actualization,physiological
1,"I don't know what to do.. I got my gf pregnant. I don't know what to do and say. I havent slept for 3 days since we found out. She's 4 weeks pregnant and already thinking about ""medical abortion"". She's not talking to me. I'm having thoughts about killing myself to escape all of this. Its the most dumb thing to do but i feel like im running out of choices. We're not financially stable. I'm a student. I should have wore a condom. What should i do.",esteem,88,0.0,0.0,1.0,0.0,0.0
2,"Every day is an effort I don't want to wake up at mornings. There will be a pile of never-ending tasks awaiting. Problems. Fix those, get new problems. Ad infinitum to the point of nausea. I've had cog-beh. therapy, I have antidepressants, I've hopped through the hoops of NLP, affirmations, binaural beats, meditation, tried to stop drinking. Why bother, I don't seem to get further than the starting point. Fucking wasted effort, money and time.",safety,75,0.0,1.0,0.0,0.0,0.0
3,Electric shock therapy and an all time low. questions. severely suicidal. My doctors want me in the hospital but I refuse to go. been there done that it doesn't help. they want to tinker with my meditation. Im considering ect as a last resort. anyone here have it done? how was it? did it help? I won't make it if I continue to feel this way. This is the worst i've ever been. I have zero hope and zero convincing reasons to want to stay alive. This depression is literally killing me.,"belongingness,safety",92,1.0,1.0,0.0,0.0,0.0
4,"I don't know why I'm depressed Lately I've been feeling really shitty and very depressed. I feel like I should be really happy, I have a new girlfriend who is great, my family is healthy. But for some reason I just feel so depressed all the time. At first I thought it was because I wasn't with my new girlfriend, because it seems to go away when ever I'm with her. But I've noticed it sometimes even when we are together. Does anyone know any ways to try and keep my mind off depression and how to feel better?","belongingness,safety,esteem,self-actualization",99,1.0,1.0,1.0,1.0,0.0
5,"Telling people I'm suicidal? I already told someone I was suicidal and was put in a mental hospital. A couple days later I was released and suicidal again. I'm still thinking of killing myself, but don't want to tell anyone because I'm just going to be put in a hospital again and come out suicidal. It sucks I have to lie to people. I want help, but not *that* kind of help.","belongingness,esteem",72,1.0,0.0,1.0,0.0,0.0


In [6]:
lab = ['belongingness','safety','esteem','self_actualization','physiological']
x = data['title_text']
y = data[lab]
print(x.shape)
print(y.shape)

(114,)
(114, 5)


In [7]:
from sklearn.model_selection import train_test_split
X, X_test, Y, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
print(X.shape)
print(X_test.shape)
print(Y.shape)
print(y_test.shape)

(91,)
(23,)
(91, 5)
(23, 5)


In [8]:
X_train, X_val ,y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=0)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(81,)
(10,)
(81, 5)
(10, 5)


# Building models

In [9]:
import pandas as pd
import transformers
from transformers import TFAutoModelForSequenceClassification, RobertaTokenizer
import re
import os
import matplotlib as mpl
import matplotlib.cm
import tensorflow as tf
import tensorflow.keras as keras
import tokenizers
from tqdm import tqdm
from typing import Optional, Union, List, Dict
from IPython.display import HTML
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from transformers import PreTrainedTokenizer
pd.set_option('display.max_colwidth', None)
# MentalBERT reference: https://huggingface.co/mental/mental-bert-base-uncased
#model_name = "mental/mental-bert-base-uncased"
#model_name = "mental/mental-roberta-base"
#model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
model_name = 'roberta-base'

In [10]:
from transformers import RobertaTokenizer, TFRobertaModel,RobertaConfig,TFRobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [11]:
print("Vocab Size:",tokenizer.vocab_size)
print()
print("Max sequence length:",tokenizer.max_len_single_sentence)
print()
print("Model Max length:",tokenizer.model_max_length )
print()
print("Padding:", tokenizer.pad)
print()
print("Max model input sizes:", tokenizer.max_model_input_sizes)

Vocab Size: 50265

Max sequence length: 510

Model Max length: 512

Padding: <bound method PreTrainedTokenizerBase.pad of PreTrainedTokenizer(name_or_path='roberta-base', vocab_size=50265, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})>

Max model input sizes: {'rober

In [12]:
train_encodingsf = tokenizer(list(X_train.values),
                                padding = "max_length",
                                truncation = True,
                                return_attention_mask = True,
                                return_token_type_ids = True,
                                return_tensors = 'np')
val_encodingsf = tokenizer(list(X_val.values),
                                padding = "max_length",
                                truncation = True,
                                return_attention_mask = True,
                                return_token_type_ids = True,
                                return_tensors = 'np')
test_encodingsf = tokenizer(list(X_test.values),
                                padding = "max_length",
                                truncation = True,
                                return_attention_mask = True,
                                return_token_type_ids = True,
                                return_tensors = 'np')

In [13]:
train_encodingsf

{'input_ids': array([[    0, 12645,  6537, ...,     1,     1,     1],
       [    0,  7682,   183, ...,     1,     1,     1],
       [    0,   100,   206, ...,     1,     1,     1],
       ...,
       [    0, 10127,    38, ...,     1,     1,     1],
       [    0, 27847,  1268, ...,     1,     1,     1],
       [    0,  2709,    10, ...,     1,     1,     1]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [14]:
print(len(train_encodingsf))

3


In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodingsf),
                                    list(y_train.values)))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodingsf),
                                    list(y_val.values)))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodingsf),
                                    list(y_test.values)))

In [16]:
from transformers import BertConfig
# Initializing a RoBERTa configuration
config = RobertaConfig()
config.output_hidden_states = False
# Initializing a model (with random weights) from the configuration
model = TFRobertaModel(config)
transformer_model = TFRobertaForSequenceClassification.from_pretrained(model_name)
# Accessing the model configuration
configuration = model.config
'''
config = BertConfig(num_labels = 200)   #dropout=0.2, attention_dropout=0.2,
config.output_hidden_states = False
transformer_model = TFAutoModelForSequenceClassification.from_pretrained(model_name,from_pt = True,config = config)
transformer_model.layers[0].trainable = True
print(transformer_model.summary())
'''

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'\nconfig = BertConfig(num_labels = 200)   #dropout=0.2, attention_dropout=0.2,\nconfig.output_hidden_states = False\ntransformer_model = TFAutoModelForSequenceClassification.from_pretrained(model_name,from_pt = True,config = config)\ntransformer_model.layers[0].trainable = True\nprint(transformer_model.summary())\n'

In [17]:
transformer_model

In [18]:
transformer_model.roberta?

In [19]:
# Reference: https://towardsdatascience.com/working-with-hugging-face-transformers-and-tf-2-0-89bf35e3555a (For BERT)
# The reference is for creating the BERT model
def create_model():
   input_token = tf.keras.layers.Input(shape=(None,), name='input_ids',dtype='int32')
   seq_token = tf.keras.layers.Input(shape=(None,), name='token_type_ids',dtype='int32')
   masked_token = tf.keras.layers.Input(shape=(None,), name='attention_mask',dtype='int32')

   embedding_layer = transformer_model.roberta(input_token,masked_token,seq_token)[0]
   #X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(embedding_layer)
   #X = tf.keras.layers.GlobalMaxPool1D()(X)
    
   X = tf.keras.layers.Dense(100, activation='relu')(embedding_layer[:,0,:])
   X = tf.keras.layers.Dropout(0.2)(X)
   X = tf.keras.layers.Dense(30, activation = 'relu',)(X)
   # Designing last layer reference: 
   # https://medium.com/deep-learning-with-keras/how-to-solve-multi-label-classification-problems-in-deep-learning-with-tensorflow-keras-7fb933243595
   X = tf.keras.layers.Dense(5, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer=tf.keras.regularizers.l2(0.01))(X)
   model = tf.keras.Model(inputs=[input_token,masked_token,seq_token], outputs = X)
   """
   for layer in model.layers[:0]:
    layer.trainable = False
   """
   return model

model = create_model()
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124055040   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]',     

In [20]:
 #['input_token', 'masked_token', 'seq_token']
    # 'input_ids', 'token_type_ids', 'attention_mask'

In [21]:
# Reference: https://towardsdatascience.com/working-with-hugging-face-transformers-and-tf-2-0-89bf35e3555a
# For implementing the BERT architecture with custom layers on top of it

In [22]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    min_delta=0,
    patience=3,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_binary_accuracy',
                                                 factor = 0.2,
                                                 patience = 1,
                                                 verbose = 1,
                                                 min_delta = 1e-4,
                                                 min_lr = 1e-11,
                                                 mode = 'max')

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = './ROBERTA',
                                                  monitor = 'val_binary_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = False,
                                                  mode = 'max')

callbacks = [earlystopping, reduce_lr, checkpointer]

In [ ]:
BATCH_SIZE = 8
N_EPOCHS = 10
optimizerr = tf.keras.optimizers.Adam(learning_rate=1e-3)
#optimizerr = tfa.optimizers.RectifiedAdam(learning_rate=5e-5)
losss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=tf.keras.metrics.binary_accuracy)
history = model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE,
          validation_data = test_dataset.shuffle(len(X_test)).batch(BATCH_SIZE),
          callbacks = callbacks)

Epoch 1/10
11/11 [==============================] - ETA: 0s - loss: 0.7982 - binary_accuracy: 0.6148
Epoch 1: val_binary_accuracy improved from -inf to 0.64348, saving model to ./ROBERTA


11/11 [==============================] - 59s 4s/step - loss: 0.7982 - binary_accuracy: 0.6148 - val_loss: 0.7310 - val_binary_accuracy: 0.6435 - lr: 0.0010


In [ ]:
trial =list(X_test)
trial_y = list(y_test)
print(len(trial))

1592


In [ ]:
def predict_proba(text_list, model, tokenizer, threshold):  
    #tokenize the text
    encodings = tokenizer(text_list,
                  padding = "max_length",
                  truncation = True,
                  return_attention_mask = True,
                  return_tensors = 'np')
    #transform to tf.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings)))
    #predict
    preds = model.predict(dataset.batch(1))
    
    #transform to array with probabilities
    #res = tf.nn.softmax(preds, axis=1).numpy()    
    #opp = tf.math.argmax(res, axis = 1)
    opp = preds>threshold
    opp = opp.astype('int')
    
    return opp

In [ ]:
y_pred = predict_proba(trial, model, tokenizer, 0.5)
print(y_pred.shape)
print(len(y_pred))

(1592, 3)
1592


In [ ]:
np.save("./y_testBERT.npy", y_test)
np.save("./y_predBERT.npy",y_pred)

In [ ]:
y_testnp =y_test.to_numpy()

def Accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / y_true.shape[0]

print("Averall Accuracy or Hamming score:",Accuracy(y_testnp, y_pred))

Averall Accuracy or Hamming score: 0.5797738693467344


In [ ]:
! zip BERT.zip ./BERT

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: BERT/ (stored 0%)


In [ ]:
import os
os.chdir(r'/kaggle/working')
r'./BERT.zip'

from IPython.display import FileLink
FileLink(r'./BERT.zip')

/kaggle/working/BERT.zip

In [ ]:
new_model = tf.keras.models.load_model('./BERT')

In [ ]:
# Evakuating the loaded model on overall test
new_model.evaluate(test_dataset.shuffle(len(X_test)).batch(BATCH_SIZE), 
               return_dict=True, 
               batch_size=BATCH_SIZE)